In [1]:
! pip install pandas
! pip install matplotlib
! pip install sklearn
! pip install seaborn

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [2]:
import pandas as pd
import sklearn.ensemble as skle
import sklearn.svm as svm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, KFold
import sklearn.metrics as sm

In [4]:
df = pd.read_csv('V5_final_dataset.csv', index_col=0).drop_duplicates()

In [5]:
X = df.drop(columns=['Glu (vein)'])
y = df['Glu (vein)']

In [6]:
y_osm = df['Osmolarity (vein)']
X_osm = df.drop(columns=['Osmolarity (vein)'])

In [7]:
X_train_o, X_test_o, y_train_o, y_test_o = train_test_split(X_osm, y_osm, test_size=0.1, random_state=42)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_train_s = preprocessing.normalize(X_train)
X_test_s = preprocessing.normalize(X_test)

In [6]:
cv = KFold(n_splits=10, shuffle=True, random_state=42)

In [7]:
params_grid_AB = {'n_estimators': range(300, 1200, 50),
                  'loss': ['linear', 'square', 'exponential']
                 }
Ada_boost = skle.AdaBoostRegressor(random_state=42)
GS_AB = GridSearchCV(Ada_boost, params_grid_AB, cv=cv, scoring='neg_mean_squared_error')
GS_AB.fit(X_train, y_train)
Ada_boost_GS_y = GS_AB.predict(X_test)

In [8]:
Ada_boost = skle.AdaBoostRegressor(random_state=42, n_estimators=1000, loss='square')
Ada_boost.fit(X_train_o, y_train_o)
Ada_boost_GS_yo = Ada_boost.predict(X_test_o)

In [9]:
sm.mean_squared_error(y_test_o, Ada_boost_GS_yo), sm.r2_score(y_test_o, Ada_boost_GS_yo)

(1.055320616093225e+30, 0.8338702437498802)

In [10]:
feat_o = {}
for i in range(len(X_train_o.columns)):
    feat_o[X_train_o.columns[i]] = Ada_boost.feature_importances_[i]
    
sorted( ((k,v) for v,k in feat_o.items()), reverse=True) 

[(0.3220126234906975, 'ctHb (vein)'),
 (0.15251927669675894, 'Cl- (vein)'),
 (0.1395901929792498, 'Na + (vein)'),
 (0.08902439878575912, 'pO2 (vein)'),
 (0.08896709646409703, 'pCO2 (vein)'),
 (0.04337559807849602, 'HCO3- (P) (vein)'),
 (0.04278355807781538, 'Ca2 + ( vein)'),
 (0.03815110739232668, 'sO2 (vein)'),
 (0.028984830410525287, 'Hct (vein)'),
 (0.01219077381532502, 'ABE (vein)'),
 (0.010985560810820706, 'Glu (vein)'),
 (0.008651710722291572, 'Lac (vein)'),
 (0.008002664181387611, 'pH (vein)'),
 (0.0046115911762977455, 'p50 (vein)'),
 (0.004585119443776119, 'ctO2 (vein)'),
 (0.0042608722272970126, 'K + (vein)'),
 (0.0012317818328148938, 'Age'),
 (7.12434142636114e-05, 'Sex'),
 (0.0, 'ctBil (vein)'),
 (0.0, 'Smoke_status'),
 (0.0, 'SBE (vein)'),
 (0.0, 'HCO3- (P, st) (vein)'),
 (0.0, 'FMetHb (vein)'),
 (0.0, 'FHbF (vein)'),
 (0.0, 'FHHb (vein)'),
 (0.0, 'FCOHb (vein)'),
 (0.0, 'F02Hb (vein)')]

In [8]:
sm.mean_squared_error(y_test, Ada_boost_GS_y), sm.r2_score(y_test, Ada_boost_GS_y)

(12.991472614721305, 0.14713787183664018)

In [11]:
GS_AB.best_estimator_, GS_AB.best_params_

(AdaBoostRegressor(loss='square', n_estimators=1000, random_state=42),
 {'loss': 'square', 'n_estimators': 1000})